In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from dotenv import load_dotenv
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from dataclasses import dataclass

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [11]:
@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [12]:
agent = create_agent(
    model=llm,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [13]:
response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Irish")
)

print(response["messages"][-1].content)

Is mise do chúntóir pearsanta. Conas is féidir liom cabhrú leat inniu?


In [14]:
response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

¡Hola! Claro que sí, con gusto te ayudaré. ¿En qué puedo asistirte hoy?


In [15]:
response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Russia")
)

print(response["messages"][-1].content)

Конечно, я буду отвечать вам только на русском языке. Чем я могу вам помочь?
